# Training a model

This notebook demonstrates how to train a model on a pre-collected dataset

In [ ]:
import os

# These environment variables control where
# training and eval logs are written.
os.environ["RUN_DIR"] = "runs"
os.environ["EVAL_RUN_DIR"] = "eval_runs"

# This is used to set a constant Tensorboard port.
os.environ["TENSORBOARD_PORT"] = str(8989)

In [ ]:
import ml.api as ml  # Source: https://github.com/codekansas/ml-starter

# Imports these files to add them to the model and task registry.
from usa.models.point2emb import Point2EmbModel
from usa.tasks.clip_sdf import ClipSdfTask

In [ ]:
# Enables logging to `stdout`.
ml.configure_logging(use_tqdm=True)

The framework used to train the models specifies six parts:

1. Model: The USA net model follows the vanilla NeRF implementation, and uses a simple MLP mapping 3D points to an output vector
2. Task: This is used to coordinate training by passing the dataset samples to the model and computing the loss function
3. Optimizer
4. Learning rate scheduler
5. Trainer
6. Loggers

In [ ]:
config = {
    "model": {
        "name": "point2emb",          # `register_model` name in `usa.models.point2emb`
        "num_layers": 4,
        "hidden_dims": 256,
        "output_dims": 513,           # CLIP = 512, SDF = 1
    },
    "task": {
        "name": "clip_sdf",           # `register_task` name in `usa.tasks.clip_sdf`
        "dataset": "lab_r3d",         # Pre-collected dataset
        "clip_model": "ViT_B_16",
        "queries": [
            "Chair",
            "Shelves",
            "Man sitting at a computer",
            "Desktop computers",
            "Wooden box",
            "Doorway",
        ],
        "rotate_image": True,         # Dataset-specific, for visualization purposes
        "finished": {
            "max_steps": 10_000,      # Number of training steps
        },
        "dataloader": {
            "train": {
                "batch_size": 16,
                "num_workers": 0,
                "persistent_workers": False,
            },
        },
    },
    "optimizer": {
        "name": "adam",
        "lr": 3e-4,
    },
    "lr_scheduler": {
        "name": "linear",
    },
    "trainer": {
        "name": "vanilla_sl",
        "exp_name": "jupyter",
        "log_dir_name": "test",
        "base_run_dir": "runs",
        "run_id": 0,
        "checkpoint": {
            "save_every_n_steps": 2500,
            "only_save_most_recent": True,
        },
        "validation": {
            "valid_every_n_steps": 250,
            "num_init_valid_steps": 1,
        },
    },
    "logger": [{"name": "tensorboard"}],
}

In [ ]:
objs = ml.instantiate_config(config)

# Unpacking the different components.
model = objs.model
task = objs.task
optimizer = objs.optimizer
lr_scheduler = objs.lr_scheduler
trainer = objs.trainer

In [ ]:
from tensorboard import notebook

# Show Tensorboard inside the notebook.
notebook.display(port=int(os.environ['TENSORBOARD_PORT']))

# Runs the training loop.
trainer.train(model, task, optimizer, lr_scheduler)